In [1]:
import os
import nd2
import napari
import pickle
import numpy as np
import pandas as pd
from skimage.io import imread
from skimage.measure import regionprops_table, profile_line
from skimage.morphology import disk,erosion
from scipy.ndimage import distance_transform_edt
from cardiomyocytes_helper_functions import segment_actin_3D,calculate_orientation,find_fibers_orientation_v2,orientations_from_vertices, signal_from_vertices,divide_cell_outside_ring

In [2]:
path_dir = r'D:\data_analysis\2022_Sahana\data\Collagen\60x images'
im_name = r'092622_ring_PDMSAp_10MCol_647-Act_561-Pax_488-Plak_DAPI_001.nd2'
path_im = os.path.join(path_dir,im_name)

path_vertices = r'D:\data_analysis\2022_Sahana\masks'

path_results = r'D:\data_analysis\2022_Sahana\results'
path_df = os.path.join(path_results,im_name.replace('.nd2','_df.pkl'))

# order of channels
    # actin
    # paxilin
    # plakoglobin
    # DAPI


## Read in the image and annotations

In [4]:
# read in image
im = nd2.imread(path_im)
im.shape

(27, 4, 512, 512)

In [5]:
# read in polygons and masks

pkl_path = os.path.join(path_vertices,im_name.replace('.nd2','_polygons.pkl'))

with open(pkl_path, 'rb') as f:
    vertices_polygons = pickle.load(f)

mask_path = os.path.join(path_vertices,im_name.replace('.nd2','_mask.png'))
mask = imread(mask_path)

## Segment actin

In [6]:
# get actin channel
image_actin = im[:,0,:,:]

# segment actin volume
image_actin_mask = segment_actin_3D(image_actin)

# flatten segmented actin
image_actin_mask_2D = np.max(image_actin_mask,axis=0)

intensity normalization: min-max normalization with NO absoluteintensity upper bound


## Calculate general properties of cells

In [7]:
# calculate properties of cells

properties = ['label','area','centroid','bbox','eccentricity','orientation','intensity_mean','intensity_image','image']

im_flat_all_channels = np.max(im,axis=0)
im_flat_all = np.append(im_flat_all_channels,np.expand_dims(image_actin_mask_2D,axis=0),axis=0)
im_flat_all = np.moveaxis(im_flat_all,0,2)

cell_measure = regionprops_table(mask, intensity_image = im_flat_all, properties = properties)
df = pd.DataFrame(cell_measure)
df['image_name'] = im_name

In [8]:
# add vertices to the table
df['vertices'] = vertices_polygons

In [10]:
df.columns

Index(['label', 'area', 'centroid-0', 'centroid-1', 'bbox-0', 'bbox-1',
       'bbox-2', 'bbox-3', 'eccentricity', 'orientation', 'intensity_mean-0',
       'intensity_mean-1', 'intensity_mean-2', 'intensity_mean-3',
       'intensity_mean-4', 'intensity_image', 'image', 'image_name',
       'vertices'],
      dtype='object')

## Characterize orientation of the cell edge (based on vertices)

In [25]:
# characterize the orientation of the edge of the cell

df['membrane_orientation'] = None
df['membrane_orientation'] = df['membrane_orientation'].astype(object)

for i,cell in df.iterrows():

    vert = cell.vertices

    membrane_orientation = orientations_from_vertices(vert)

    df.at[i,'membrane_orientation'] = membrane_orientation


## Calculate general flow of actin

In [26]:
# characterize actin orientation in the cells

df['actin_detected'] = None
df['actin_detected'] = df['actin_detected'].astype(object)
df['actin_angles'] = None
df['actin_angles'] = df['actin_angles'].astype(object)

for i,cell in df.iterrows():

    im_single_cell_actin = cell.intensity_image[:,:,4]

    actin,actin_angles = find_fibers_orientation_v2(im_single_cell_actin)
    
    # calculate mean orientation of the fibers
    actin_orientation = np.median(actin_angles)

    # calculate spread of orientations
    actin_spread = np.std(actin_angles)

    df.at[i,'actin_detected'] = np.array(actin)
    df.at[i,'actin_angles'] = np.array(actin_angles)
    df.loc[i,'actin_orientation'] = actin_orientation
    df.loc[i,'actin_spread'] = actin_spread

## Characterize signals at the edge

In [27]:
signal_max = np.max(im[:,2,:,:],axis=0)

df['plak_signal'] = None
df['plak_signal'] = df['plak_signal'].astype(object)

for i,cell in df.iterrows():

    vert = cell.vertices

    signal_line = signal_from_vertices(vert,signal_max)

    df.at[i,'plak_signal'] = signal_line

## Generate edge regions

In [28]:
ring_thickness = 10
segment_number = 25

df['outside_ring_regions'] = None

for i,cell in df.iterrows():

    cell_image = cell.image

    cell_center = [cell['centroid-1']-cell['bbox-1'],cell['centroid-0']-cell['bbox-0']]

    points_array = divide_cell_outside_ring(cell_image,cell_center,ring_thickness,segment_number)

    # store in the data frame
    df.at[i,'outside_ring_regions'] = points_array

KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.
KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.


## Calculate signal based on edge regions

In [29]:
# initiate columns
for s in range(4):
    
    df[f'signal_edge_{str(s).zfill(2)}'] = None

for i,cell in df.iterrows():

    # get positions of outside points
    points_array = cell['outside_ring_regions']

    for s in range(4):

        signal_image = cell.intensity_image[:,:,s]

        signal_list = []

        for x in range(segment_number):

            points_selected = points_array[points_array[:,2]==x,:]

            signals = [signal_image[x[0],x[1]] for x in np.array([points_selected[:,0],points_selected[:,1]]).T]

            signal = np.mean(signals)

            signal_list.append(signal)

        df.at[i,f'signal_edge_{str(s).zfill(2)}'] = signal_list

## Calculate radial distributions

In [30]:
layer_num = 20

# initiate columns
for s in range(4):
    
    df[f'signal_radial_{str(s).zfill(2)}'] = None

for i,cell in df.iterrows():

    # generate distance transform
    cell_shape = cell.image
    dist = distance_transform_edt(cell_shape)

    # digitize distance transform
    step = np.max(dist)/layer_num

    for n in range(layer_num):

        dist[(dist>(n*step)) & (dist<=((n+1)*step))] = n+1

    # calculate radial distribution signals
    for s in range(4):

        signal_image = cell.intensity_image[:,:,s]

        signal_list = []

        for n in range(layer_num):

            signal = np.mean(signal_image*(dist==(n+1)))

            signal_list.append(signal)

        df.at[i,f'signal_radial_{str(s).zfill(2)}'] = signal_list


## Show data frame

In [31]:
df

,label,area,centroid-0,centroid-1,bbox-0,bbox-1,bbox-2,bbox-3,eccentricity,orientation,...,plak_signal,outside_ring_regions,signal_edge_00,signal_edge_01,signal_edge_02,signal_edge_03,signal_radial_00,signal_radial_01,signal_radial_02,signal_radial_03
0,1,21500,416.317721,305.246605,351,191,512,448,0.927055,-1.159706,...,"[161.0, 227.0, 234.0, 219.0, 216.0, 210.0, 210...","[[0, 197, 20], [0, 198, 20], [0, 199, 20], [0,...","[487.13100436681225, 934.7450980392157, 1447.5...","[446.30131004366814, 660.8823529411765, 659.57...","[400.69868995633186, 230.50980392156862, 302.3...","[146.6943231441048, 148.9607843137255, 152.627...","[43.7223578316456, 46.59869009353022, 61.15134...","[21.522029146627354, 21.356164052492932, 25.74...","[15.513594509026754, 18.263093022693766, 23.39...","[5.059163303284433, 5.015032505981584, 6.48582..."
1,2,17456,326.136801,245.599736,269,135,391,353,0.863103,-1.429548,...,"[1357.0, 1275.0, 1378.0, 1378.0, 1454.0, 1527....","[[0, 108, 18], [0, 109, 18], [0, 110, 18], [0,...","[1345.6129032258063, 1315.0406091370558, 1729....","[652.883064516129, 901.5228426395939, 731.8295...","[264.09274193548384, 299.6852791878173, 425.71...","[149.31451612903226, 153.55329949238578, 155.0...","[66.7315761768687, 88.44051737103324, 74.47289...","[25.995676041510002, 33.55004511956685, 29.377...","[28.75045119566852, 34.718115506091145, 26.360...","[6.841442322153707, 8.951195668521581, 7.74936..."
2,3,6480,219.750309,130.168056,148,98,279,171,0.860131,0.197693,...,"[256.0, 245.0, 218.0, 229.0, 241.0, 235.0, 214...","[[0, 31, 18], [1, 30, 18], [1, 31, 18], [2, 29...","[2892.4157303370785, 1846.7439024390244, 2205....","[835.7415730337078, 639.8658536585366, 727.796...","[368.96629213483146, 238.26829268292684, 375.8...","[172.65168539325842, 166.6341463414634, 152.05...","[93.46658998222316, 113.636934016522, 114.7067...","[26.15162605876817, 31.04695179337028, 31.6036...","[9.857785213845029, 12.410436055631077, 11.066...","[6.096413259437415, 7.270208093694447, 6.89114..."
3,4,17279,221.896348,231.103131,141,144,291,332,0.792619,1.247677,...,"[502.0, 470.0, 413.0, 327.0, 281.0, 336.0, 496...","[[0, 9, 15], [0, 10, 15], [0, 11, 15], [0, 12,...","[1218.0708661417323, 1587.5771028037384, 1608....","[581.9409448818898, 646.2663551401869, 565.186...","[407.248031496063, 457.9789719626168, 481.2217...","[148.98425196850394, 148.76401869158877, 158.2...","[64.24659574468085, 85.42234042553191, 74.4386...","[22.763297872340427, 31.481063829787235, 27.03...","[26.648687943262413, 31.45613475177305, 23.964...","[6.66677304964539, 8.624716312056737, 7.063687..."


## Save data frame

In [32]:
df.to_pickle(path_df)

## Visualize image

In [12]:
viewer = napari.Viewer()
viewer.add_image(im[:,0,:,:],blending='additive',colormap='magenta')
viewer.add_image(signal_max,blending='additive',colormap='green')
viewer.add_image(image_actin_mask,blending='additive',colormap='green',visible=False)
viewer.add_image(image_actin_mask_2D,blending='additive',colormap='gray')
viewer.add_labels(mask,blending='additive')

Assistant skips harvesting pyclesperanto as it's not installed.


<Labels layer 'mask' at 0x25aa4d19df0>